In [17]:
!pip install transformers
!pip install datasets #dasets de huggin face

#IMPORTANTE trabajar con las librerias actualizadas o que sean compatibles con el codigo
!pip install --upgrade datasets
!pip install --upgrade transformers


In [18]:
import datasets
import transformers

In [ ]:
#es el dataloader para descargar el datasets
from datasets import load_dataset
reviews = load_dataset('rotten_tomatoes')
reviews

In [ ]:
#¿como pasamos el dataset de huggin Face a pandas?
df_train = reviews['train'].to_pandas()
df_train

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
8525,any enjoyment will be hinge from a personal th...,0
8526,if legendary shlockmeister ed wood had ever ma...,0
8527,hardly a nuanced portrait of a young woman's b...,0
8528,"interminably bleak , to say nothing of boring .",0


In [ ]:
# @title score

from matplotlib import pyplot as plt
import seaborn as sns
df_train.groupby('score').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
#PIPELINE es la manera principal de interactuar con los transformadores
from transformers import pipeline
#creamos un clasificador y ñe decimos que es un clasificador de sentimientos
#dentro de pipeline podemos decirle que tipo de MODELO vamos a utilizar
#en el ejemplo, como no especificamos el modelos, se utilizara no por defecto
clasificador = pipeline('sentiment-analysis')
clasificador('I love you')

In [ ]:
#ejemplo de como aplicar el clasificador al dataset
def score(review_text):
  return clasificador(review_text)[0]['label']

#aplicamos la funcion Score a la columna text y agregamos el resultado como una nueva columna
df_train['score'] = df_train['text'].apply(score)
df_train

In [ ]:
#para logearnos a huggin face desde la notebook
from huggingface_hub import notebook_login

notebook_login()

# NPL Pipelines

In [ ]:
help(pipeline)
#las pipeline son sencillas de usar, eliges el modelo y la tarea que quieres que realize
#con help vemos el listado de las posibles tareas que acepta pipeline


Help on function pipeline in module transformers.pipelines:

pipeline(task: Optional[str] = None, model: Union[str, ForwardRef('PreTrainedModel'), ForwardRef('TFPreTrainedModel'), NoneType] = None, config: Union[str, transformers.configuration_utils.PretrainedConfig, NoneType] = None, tokenizer: Union[str, transformers.tokenization_utils.PreTrainedTokenizer, ForwardRef('PreTrainedTokenizerFast'), NoneType] = None, feature_extractor: Union[str, ForwardRef('SequenceFeatureExtractor'), NoneType] = None, image_processor: Union[str, transformers.image_processing_utils.BaseImageProcessor, NoneType] = None, processor: Union[str, transformers.processing_utils.ProcessorMixin, NoneType] = None, framework: Optional[str] = None, revision: Optional[str] = None, use_fast: bool = True, token: Union[str, bool, NoneType] = None, device: Union[int, str, ForwardRef('torch.device'), NoneType] = None, device_map=None, torch_dtype=None, trust_remote_code: Optional[bool] = None, model_kwargs: Optional[Dict[s

# Tokenizacion

In [ ]:
from transformers import AutoTokenizer
#un AutoTokenaizer necesita leer un tokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')
#los LLM tienen un tokenizador dentro de ellos

In [9]:
#tenemos un texto para tokenizar
oracion = "Preposterous, I'm flabbergasted!"

input_ids = tokenizer(oracion, return_tensors = 'pt').input_ids
#Return_tensors = 'pt' quiere decir que el tokenizador debe devolver los token como tenzores de Torch

input_ids

#Palabra --> tokens --> ids unicos --> vertor embed



tensor([[37534,  6197,   516,    11,   314,  1101,   781,   397,  3900,  8992,
             0]])

como convertimos los tokens a texto?

In [10]:
tokenizer.decode(37534)

'Prep'

In [ ]:
for token_id in input_ids[0]:
  print(tokenizer.decode(token_id))

si quisieramos un modelo para que complete la oracion

In [26]:
from transformers import AutoModelForCausalLM
#utilizamos AotuModel para no recordar cada modelo con su respectivo tokenizer

gpt2 = AutoModelForCausalLM.from_pretrained('gpt2')
texto = "I skip across the"
input_ids = tokenizer(texto, return_tensors = 'pt').input_ids
#Return_tensors = 'pt' quiere decir que el tokenizador debe devolver los token como tenzores de Torch

input_ids

#Palabra --> tokens --> ids unicos --> vertor embed

tensor([[   40, 14267,  1973,   262]])

In [29]:
outputs = gpt2(input_ids)
outputs.logits.shape
#logits es como un softmax
#vamos a ver que torch.Size([1, 4, 50257])
#1 es el número de lote que
#4 es el número de tokens
#50257 es el tmaño total del vocavulario gpt2




torch.Size([1, 4, 50257])